In [10]:
import pandas as pd
import os
import git
import numpy as np

def get_git_root(path):

        git_repo = git.Repo(path, search_parent_directories=True)
        #git_root = git_repo.git.rev_parse("--show-toplevel")
        
        return git_repo.working_dir

In [11]:
top_level_git_dir = get_git_root(os.getcwd())
raw_data_dir = os.path.join(top_level_git_dir, "data", "raw")

train_csv_path = os.path.join(raw_data_dir, "train_values.csv")
test_csv_path = os.path.join(raw_data_dir, "test_values.csv")
train_labels_csv_path = os.path.join(raw_data_dir, "train_labels.csv")
submission_format_csv_path = os.path.join(raw_data_dir, "submission_format.csv")

train_df = pd.read_csv(train_csv_path, index_col = "row_id")
train_labels_df = pd.read_csv(train_labels_csv_path, index_col = "process_id")
test_df = pd.read_csv(test_csv_path, index_col = "row_id")
submission_format_csv_path = pd.read_csv(submission_format_csv_path, index_col = "process_id")


/Users/greg.bolla/.local/share/virtualenvs/driven-data-rinse-over-run-w5STMFPe/lib/python3.6/site-packages/numpy/lib/arraysetops.py:571: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [15]:
response_dict = {}
for proc_id in train_df.process_id.unique():
    final_phase_for_proc = train_df[(train_df.process_id == proc_id) & (train_df.target_time_period == True)]
    final_phase_for_proc = final_phase_for_proc.assign(final_phase_turbidity = \
        np.maximum(0, final_phase_for_proc.return_flow) * final_phase_for_proc.return_turbidity)
    proc_id_response = final_phase_for_proc["final_phase_turbidity"].sum()
    
    response_dict[proc_id] = proc_id_response

In [16]:
response_dict

{20001: 4318275.12554818,
 20002: 437528.57109794125,
 20003: 427197.73393308267,
 20004: 719783.0261053684,
 20005: 413310.6503749337,
 20008: 4100847.0864118547,
 20011: 3058669.195203994,
 20014: 2706871.4373974237,
 20016: 521755.33349069883,
 20017: 103724.28729467509,
 20018: 1366681.8465563094,
 20019: 53408.509967789665,
 20023: 1875568.3397814478,
 20025: 327966.6135227812,
 20030: 508890.15811140987,
 20033: 668203.1933072086,
 20035: 433945.2282390981,
 20037: 314885.3107635207,
 20039: 1763055.4335154223,
 20042: 689355.9961675403,
 20043: 1585300.1251132558,
 20044: 349528.2890806126,
 20045: 983082.6895779017,
 20046: 1079254.346558409,
 20047: 326358.14896491176,
 20049: 660466.9393707704,
 20050: 553740.272660489,
 20051: 59006.66805360914,
 20053: 1707946.5172790927,
 20057: 758868.590742599,
 20058: 58417.41697918879,
 20062: 589065.7840487394,
 20065: 692860.5903703257,
 20066: 557125.276612133,
 20067: 10079370.048694504,
 20070: 454609.2506713714,
 20071: 25770.556

In [24]:
train_df = train_df.assign(turbidity_in_liters = \
                np.maximum(0, train_df.return_flow) * train_df.return_turbidity)


In [26]:
ts_cols = [
    'process_id',
    'supply_flow',
    'supply_pressure',
    'return_temperature',
    'return_conductivity',
    'return_turbidity',
    'return_flow',
    'tank_level_pre_rinse',
    'tank_level_caustic',
    'tank_level_acid',
    'tank_level_clean_water',
    'tank_temperature_pre_rinse',
    'tank_temperature_caustic',
    'tank_temperature_acid',
    'tank_concentration_caustic',
    'tank_concentration_acid',
    "turbidity_in_liters"
]

def prep_time_series_features(df, columns = None):
    
    if columns is None:
        columns = df.columns
    
    df = df.sort_values(by=["process_id", "timestamp"], ascending=True)
    ts_df = df[ts_cols].set_index('process_id')
    
    # define fxn before calling in .agg to make col name more descriptive (in place of <lambda>)
    def last_five_mean(x):
        return x.tail(5).mean()
    
    def phase_duration(x):
        x[0]
    
    ts_features_df = ts_df.groupby('process_id').agg(['min', 'max', 'mean', 'std', last_five_mean])
    
    return ts_features_df

In [35]:
train_df.sort_values(by=["process_id", "timestamp"]).head(n=28)

,process_id,object_id,phase,timestamp,pipeline,supply_flow,supply_pressure,return_temperature,return_conductivity,return_turbidity,...,tank_temperature_caustic,tank_temperature_acid,tank_concentration_caustic,tank_concentration_acid,tank_lsh_caustic,tank_lsh_acid,tank_lsh_clean_water,tank_lsh_pre_rinse,target_time_period,turbidity_in_liters
row_id,,,,,,,,,,,,,,,,,,,,,
0,20001,405,pre_rinse,2018-04-15 04:20:47,L4,8550.348,0.615451,18.044704,4.990765,0.177228,...,83.036750,73.032410,45.394646,44.340126,False,0.0,False,0.0,False,2796.110205
1,20001,405,pre_rinse,2018-04-15 04:20:49,L4,11364.294,0.654297,18.229168,3.749680,0.122975,...,83.015045,73.032410,45.394447,44.339380,False,0.0,False,0.0,False,1628.362812
2,20001,405,pre_rinse,2018-04-15 04:20:51,L4,12174.479,0.699870,18.395544,2.783954,0.387008,...,83.015045,73.032410,45.396280,44.336735,False,0.0,False,0.0,False,4140.516455
3,20001,405,pre_rinse,2018-04-15 04:20:53,L4,13436.776,0.761502,18.583622,1.769353,0.213397,...,83.036750,73.032410,45.401875,44.333110,False,0.0,False,0.0,False,1708.843164
4,20001,405,pre_rinse,2018-04-15 04:20:55,L4,13776.766,0.837240,18.627026,0.904020,0.148293,...,83.015045,73.032410,45.398197,44.334373,False,0.0,False,0.0,False,890.357654
5,20001,405,pre_rinse,2018-04-15 04:20:57,L4,18710.215,0.944444,18.583622,0.509684,0.166377,...,82.993350,73.032410,45.385914,44.336000,False,0.0,False,0.0,False,818.406942
6,20001,405,pre_rinse,2018-04-15 04:20:59,L4,21216.725,1.032118,18.460648,0.443773,0.292969,...,82.993350,73.032410,45.369984,44.332523,False,0.0,False,0.0,False,1813.040830
7,20001,405,pre_rinse,2018-04-15 04:21:01,L4,22403.066,1.087891,18.420862,0.521089,0.271267,...,82.953560,73.032410,45.381107,44.336490,False,0.0,False,0.0,False,2063.350718
8,20001,405,pre_rinse,2018-04-15 04:21:03,L4,23278.357,1.140842,18.420862,0.779835,0.361690,...,82.953560,73.032410,45.402290,44.334263,False,0.0,False,0.0,False,3105.655616


In [34]:
train_df.sort_values(by=["process_id", "timestamp"]).groupby('process_id')

,process_id,object_id,phase,timestamp,pipeline,supply_flow,supply_pressure,return_temperature,return_conductivity,return_turbidity,...,tank_temperature_caustic,tank_temperature_acid,tank_concentration_caustic,tank_concentration_acid,tank_lsh_caustic,tank_lsh_acid,tank_lsh_clean_water,tank_lsh_pre_rinse,target_time_period,turbidity_in_liters
row_id,,,,,,,,,,,,,,,,,,,,,
0,20001,405,pre_rinse,2018-04-15 04:20:47,L4,8550.3480,0.615451,18.044704,4.990765,0.177228,...,83.036750,73.032410,45.394646,44.340126,False,0.0,False,0.0,False,2796.110205
1,20001,405,pre_rinse,2018-04-15 04:20:49,L4,11364.2940,0.654297,18.229168,3.749680,0.122975,...,83.015045,73.032410,45.394447,44.339380,False,0.0,False,0.0,False,1628.362812
2,20001,405,pre_rinse,2018-04-15 04:20:51,L4,12174.4790,0.699870,18.395544,2.783954,0.387008,...,83.015045,73.032410,45.396280,44.336735,False,0.0,False,0.0,False,4140.516455
3,20001,405,pre_rinse,2018-04-15 04:20:53,L4,13436.7760,0.761502,18.583622,1.769353,0.213397,...,83.036750,73.032410,45.401875,44.333110,False,0.0,False,0.0,False,1708.843164
4,20001,405,pre_rinse,2018-04-15 04:20:55,L4,13776.7660,0.837240,18.627026,0.904020,0.148293,...,83.015045,73.032410,45.398197,44.334373,False,0.0,False,0.0,False,890.357654
5,20001,405,pre_rinse,2018-04-15 04:20:57,L4,18710.2150,0.944444,18.583622,0.509684,0.166377,...,82.993350,73.032410,45.385914,44.336000,False,0.0,False,0.0,False,818.406942
6,20001,405,pre_rinse,2018-04-15 04:20:59,L4,21216.7250,1.032118,18.460648,0.443773,0.292969,...,82.993350,73.032410,45.369984,44.332523,False,0.0,False,0.0,False,1813.040830
7,20001,405,pre_rinse,2018-04-15 04:21:01,L4,22403.0660,1.087891,18.420862,0.521089,0.271267,...,82.953560,73.032410,45.381107,44.336490,False,0.0,False,0.0,False,2063.350718
8,20001,405,pre_rinse,2018-04-15 04:21:03,L4,23278.3570,1.140842,18.420862,0.779835,0.361690,...,82.953560,73.032410,45.402290,44.334263,False,0.0,False,0.0,False,3105.655616


In [32]:
train_df.sort_values(by=["process_id", "timestamp"], ascending=False).groupby('process_id').head()

,process_id,object_id,phase,timestamp,pipeline,supply_flow,supply_pressure,return_temperature,return_conductivity,return_turbidity,...,tank_temperature_caustic,tank_temperature_acid,tank_concentration_caustic,tank_concentration_acid,tank_lsh_caustic,tank_lsh_acid,tank_lsh_clean_water,tank_lsh_pre_rinse,target_time_period,turbidity_in_liters
row_id,,,,,,,,,,,,,,,,,,,,,
5987819,27989,921,final_rinse,2018-03-20 16:49:55,L7,0.000000,0.008030,11.367911,0.346906,0.115741,...,82.505066,72.214990,45.027397,44.473400,False,0.0,False,0.0,True,0.837245
5987818,27989,921,final_rinse,2018-03-20 16:49:53,L7,-14.467592,0.008030,11.367911,0.346918,0.126591,...,82.505066,72.214990,45.043976,44.473915,False,0.0,False,0.0,True,1.144671
5987817,27989,921,final_rinse,2018-03-20 16:49:51,L7,0.000000,0.008030,11.385995,0.347310,0.094039,...,82.505066,72.214990,45.063404,44.474487,False,0.0,False,0.0,True,0.680261
5987816,27989,921,final_rinse,2018-03-20 16:49:49,L7,-14.467592,0.008030,11.389612,0.347658,0.112124,...,82.505066,72.214990,45.091305,44.475445,False,0.0,False,0.0,True,1.013851
5987815,27989,921,final_rinse,2018-03-20 16:49:47,L7,-14.467592,0.008247,11.389612,0.347515,0.122975,...,82.483360,72.214990,45.106262,44.474274,False,0.0,False,0.0,True,1.111966
5986947,27988,944,final_rinse,2018-04-25 09:21:02,L9,3.616898,0.047526,14.554398,0.351493,0.481047,...,82.411026,72.504340,45.036182,45.745193,False,0.0,False,0.0,True,4.349749
5986946,27988,944,final_rinse,2018-04-25 09:21:00,L9,3.616898,0.047526,14.554398,0.351590,0.415943,...,82.432724,72.489876,45.036427,45.708405,False,0.0,False,0.0,True,3.761061
5986945,27988,944,final_rinse,2018-04-25 09:20:58,L9,3.616898,0.047526,14.554398,0.351261,0.415943,...,82.436340,72.460940,45.034527,45.714737,False,0.0,False,0.0,True,3.761061
5986944,27988,944,final_rinse,2018-04-25 09:20:56,L9,3.616898,0.047526,14.554398,0.351382,0.426794,...,82.432724,72.421150,45.036922,45.749603,False,0.0,False,0.0,True,3.859176
